In [1]:
import numpy as np
import pandas as pd

In [2]:
air_reserve = pd.read_csv("data/raw/air_reserve.csv")
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [3]:
hpg_reserve = pd.read_csv("data/raw/hpg_reserve.csv")
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


<h3> Preprocessing Visitors Reservation Data </h3>
Initially joining HPG reservation data with air-hpg-joined data to get air_store_id for each restaurant
<h4> Feature Engineering </h4>
<p> 1. Time Between reservation to visit </p>
<h4> Aggregations </h4>
<p> 1. Total number of guests reserved </p>
<p> 2. Average time gap between reserve and visit </p>


In [4]:
air_hpg_join = pd.read_csv('data/raw/store_id_relation.csv')
hpg_reserve_merged = pd.merge(hpg_reserve, air_hpg_join, on='hpg_store_id', how="inner")
hpg_reserve_merged = hpg_reserve_merged.drop('hpg_store_id', axis=1)

In [5]:
reservation_data = pd.concat([air_reserve, hpg_reserve_merged])
reservation_data.head()

,air_store_id,reserve_datetime,reserve_visitors,visit_datetime
0,air_877f79706adbfb06,2016-01-01 16:00:00,1,2016-01-01 19:00:00
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,3,2016-01-01 19:00:00
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,6,2016-01-01 19:00:00
3,air_877f79706adbfb06,2016-01-01 16:00:00,2,2016-01-01 20:00:00
4,air_db80363d35f10926,2016-01-01 01:00:00,5,2016-01-01 20:00:00


In [6]:
def preprocess_reservation_data(reserve_df):
    reserve_df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'])
    reserve_df['visit_datetime'] = pd.to_datetime(reserve_df['visit_datetime'])
    reserve_df['visit_date'] = reserve_df['visit_datetime'].dt.date
    reserve_df['reserve_visit_gap'] = reserve_df['visit_datetime'] - reserve_df['reserve_datetime']

preprocess_reservation_data(reservation_data)
reservation_data.head()

,air_store_id,reserve_datetime,reserve_visitors,visit_datetime,visit_date,reserve_visit_gap
0,air_877f79706adbfb06,2016-01-01 16:00:00,1,2016-01-01 19:00:00,2016-01-01,03:00:00
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,3,2016-01-01 19:00:00,2016-01-01,00:00:00
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,6,2016-01-01 19:00:00,2016-01-01,00:00:00
3,air_877f79706adbfb06,2016-01-01 16:00:00,2,2016-01-01 20:00:00,2016-01-01,04:00:00
4,air_db80363d35f10926,2016-01-01 01:00:00,5,2016-01-01 20:00:00,2016-01-01,19:00:00


In [7]:
def aggregate_reservation_data(reserve_df):
    aggs = {'reserve_visitors': 'sum'}
    group_cols = ['air_store_id', 'visit_date']
    reserve_group_df = reserve_df.groupby(group_cols, as_index=False).agg(aggs)
    reserve_group_df.reset_index(inplace=True, drop=True)
    reserve_group_df = reserve_group_df.sort_values(['air_store_id', 'visit_date']).reset_index(drop=True)
    return reserve_group_df

reservation_group = aggregate_reservation_data(reservation_data)
reservation_group.visit_date = pd.to_datetime(reservation_group.visit_date)
reservation_group.head()

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-01-14,2
1,air_00a91d42b08b08d9,2016-01-15,4
2,air_00a91d42b08b08d9,2016-01-16,2
3,air_00a91d42b08b08d9,2016-01-22,2
4,air_00a91d42b08b08d9,2016-01-29,5


In [8]:
air_visit = pd.read_csv('data/raw/air_visit_data.csv')
air_visit.visit_date = pd.to_datetime(air_visit.visit_date)
air_visit = air_visit.sort_values(['air_store_id', 'visit_date'])
air_visit.head()

,air_store_id,visit_date,visitors
87534,air_00a91d42b08b08d9,2016-07-01,35
87535,air_00a91d42b08b08d9,2016-07-02,9
87536,air_00a91d42b08b08d9,2016-07-04,20
87537,air_00a91d42b08b08d9,2016-07-05,25
87538,air_00a91d42b08b08d9,2016-07-06,29


In [9]:
dates_info = pd.read_csv('data/raw/date_info.csv')
dates_info.calendar_date = pd.to_datetime(dates_info.calendar_date)
dates_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [10]:
def merge_complete_data(visits_info, reserve_info, dates_info):
    visits_reserve_info = visits_info.merge(reserve_info, on=['air_store_id', 'visit_date'], how='left')
    visits_reserve_dates = visits_reserve_info.merge(dates_info, left_on='visit_date', right_on='calendar_date')
    
    # Filling 0s in place of NAs assuming there were no reserved guests on that date
    visits_reserve_dates = visits_reserve_dates.fillna(0)
    return visits_reserve_dates

complete_train_data = merge_complete_data(air_visit, reservation_group, dates_info)
complete_train_data.head()

,air_store_id,visit_date,visitors,reserve_visitors,calendar_date,day_of_week,holiday_flg
0,air_00a91d42b08b08d9,2016-07-01,35,1.0,2016-07-01,Friday,0
1,air_0241aa3964b7f861,2016-07-01,10,0.0,2016-07-01,Friday,0
2,air_034a3d5b40d5b1b1,2016-07-01,19,0.0,2016-07-01,Friday,0
3,air_036d4f1ee7285390,2016-07-01,37,0.0,2016-07-01,Friday,0
4,air_03963426c9312048,2016-07-01,55,0.0,2016-07-01,Friday,0


In [11]:
test_data = pd.read_csv('data/raw/sample_submission.csv')
test_data['visit_date'] = test_data['id'].map(lambda x: str(x).split('_')[2])
test_data['air_store_id'] = test_data['id'].map(lambda x: '_'.join(x.split('_')[:2]))
test_data['visit_date'] = pd.to_datetime(test_data['visit_date'])
test_data = test_data.drop(['id', 'visitors'], axis=1)
complete_test_data = merge_complete_data(test_data, reservation_group, dates_info)

In [12]:
complete_test_data = complete_test_data[['air_store_id', 'visit_date', 'reserve_visitors', 'calendar_date', 'day_of_week', 'holiday_flg']]
complete_test_data.head()

,air_store_id,visit_date,reserve_visitors,calendar_date,day_of_week,holiday_flg
0,air_00a91d42b08b08d9,2017-04-23,0.0,2017-04-23,Sunday,0
1,air_0164b9927d20bcc3,2017-04-23,0.0,2017-04-23,Sunday,0
2,air_0241aa3964b7f861,2017-04-23,0.0,2017-04-23,Sunday,0
3,air_0328696196e46f18,2017-04-23,0.0,2017-04-23,Sunday,0
4,air_034a3d5b40d5b1b1,2017-04-23,0.0,2017-04-23,Sunday,0


<h4>Storing test train files for further use</h4>

In [13]:
complete_train_data.to_csv('data/processed/train_proc_1.csv', index=False)
complete_test_data.to_csv('data/processed/test_proc_1.csv', index=False)